In [1]:
import os
from pathlib import Path

cwd = Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent)

#### Load Dataset

In [2]:
from EmoBox.EmoBox import EmoDataset, EmoEval

In [ ]:
dataset = "ravdess"
fold = 1  # different datasets have different number of folds, which can be find in data/
user_data_dir = "./" # path to EmoBox - FIXED: Changed from "Emobox" to "EmoBox"
meta_data_dir = "EmoBox/data/" # path to data folder - FIXED: Changed from "Emobox" to "EmoBox"

train = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="train")
test = EmoDataset(dataset, user_data_dir, meta_data_dir, fold=fold, split="test")

since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 890 samples, only 890 exists in data dir EmoBox/data/
load in 291 samples, only 291 exists in data dir EmoBox/data/
Num. training samples 890
Num. valid samples 0
Num. test samples 291
Using label_map {'Disgust': 'Disgust', 'Excited': 'Excited', 'Surprise': 'Surprise', 'Sad': 'Sad', 'Sarcastic': 'Sarcastic', 'Happy': 'Happy', 'Neutral': 'Neutral', 'Angry': 'Angry'}
since there is no official valid data, use random split for train valid split, with a ratio of [80, 20]
load in 890 samples, only 890 exists in data dir EmoBox/data/
load in 291 samples, only 291 exists in data dir EmoBox/data/
Num. training samples 890
Num. valid samples 0
Num. test samples 291
Using label_map {'Disgust': 'Disgust', 'Excited': 'Excited', 'Surprise': 'Surprise', 'Sad': 'Sad', 'Sarcastic': 'Sarcastic', 'Happy': 'Happy', 'Neutral': 'Neutral', 'Angry': 'Angry'}


In [4]:
sample = test[0]
sample

/nfs/home/dasaro/research/FairMLLM-Emotion-Recognition/EmoBox/EmoBox/EmoDataset.py:154: UserWarning: PySoundFile failed. Trying audioread instead.
  wav, sr = librosa.load(wav_path, sr=None, mono=mono)
/nfs/home/dasaro/research/FairMLLM-Emotion-Recognition/.venv/lib/python3.10/site-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


NoBackendError: 

In [5]:
test.label_map.values()

dict_values(['Angry', 'Surprise', 'Sad', 'Disgust', 'Happy', 'Calm', 'Neutral', 'Fear'])

In [6]:
from collections import Counter
labels =  [data['label'] for data in test]
Counter(labels)

Counter({'Disgust': 32,
         'Fear': 32,
         'Surprise': 32,
         'Calm': 32,
         'Happy': 32,
         'Angry': 32,
         'Sad': 32,
         'Neutral': 16})

#### Load Model

In [7]:
import torch
from mllm_emotion_classifier.models import ModelFactory

device = "cuda" if torch.cuda.is_available() else "cpu"
model = ModelFactory.create(
    name="qwen2-audio-instruct",
    class_labels=set(train.label_map.values()),
    do_sample=False,
    prompt_name="user_labels",
    device=device
)

/nfs/home/dasaro/research/FairMLLM-Emotion-Recognition/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████████████████████████████████████████████████| 876/876 [00:02<00:00, 297.49it/s, Materializing param=multi_modal_projector.linear.weight]


In [8]:
data_loader = torch.utils.data.DataLoader(
    dataset=test,
    batch_size=4,
    num_workers=4,
    pin_memory=True,
    drop_last=False,
    collate_fn=model.collate_fn
)

In [ ]:
# from tqdm import tqdm
# predictions, labels = [], []
# i = 0
# for inputs, lbl in tqdm(data_loader, total=len(data_loader)):
#     inputs = {k: v.to(model.device) for k, v in inputs.items()}
#     preds = model.predict(inputs)
#     predictions.extend(preds)
#     labels.extend(lbl)
#     i += 1
#     if i == 100: break

#### Evaluation

In [10]:
from mllm_emotion_classifier.evaluate import Evaluator

evaluator = Evaluator()
evaluator.evaluate(model, data_loader)


Evaluating qwen2-audio-instruct on ravdess


Inference: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:26<00:00,  2.24it/s]


{'timestamp': '2026-01-01 20:08:47',
 'dataset': 'ravdess',
 'model_name': 'qwen2-audio-instruct',
 'fold': None,
 'num_samples': 240,
 'valid_rate': 1.0,
 'class_labels': ['Happy',
  'Calm',
  'Disgust',
  'Fear',
  'Angry',
  'Surprise',
  'Neutral',
  'Sad'],
 'metrics': {'global': {'f1_macro': 0.5797,
   'f1_weighted': 0.5798,
   'accuracy_unweighted': 0.6367,
   'accuracy_weighted': 0.6333,
   'precision_macro': 0.6369,
   'precision_weighted': 0.646,
   'recall_macro': 0.6367,
   'recall_weighted': 0.6333},
  'classwise': {'accuracy': {'Angry': 0.975,
    'Calm': 0.8667,
    'Disgust': 0.8708,
    'Fear': 0.9333,
    'Happy': 0.9583,
    'Neutral': 0.9333,
    'Sad': 0.8375,
    'Surprise': 0.8917},
   'false_positive_rate': {'Angry': 0.0,
    'Calm': 0.0,
    'Disgust': 0.149,
    'Fear': 0.0385,
    'Happy': 0.0144,
    'Neutral': 0.0491,
    'Sad': 0.1683,
    'Surprise': 0.0},
   'false_negative_rate': {'Angry': 0.1875,
    'Calm': 1.0,
    'Disgust': 0.0,
    'Fear': 0.25,
 

In [11]:
evaluator.results['metrics']['global']

{'f1_macro': 0.5797,
 'f1_weighted': 0.5798,
 'accuracy_unweighted': 0.6367,
 'accuracy_weighted': 0.6333,
 'precision_macro': 0.6369,
 'precision_weighted': 0.646,
 'recall_macro': 0.6367,
 'recall_weighted': 0.6333}